In [6]:
!pip install category_encoders

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import tensorflow as tf
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
import math
import pickle

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [2]:
def wrangle(X):

  X = X.filter(['log_price', 'room_type', 'accommodates', 'bathrooms', 'latitude',
                'longitude','review_scores_rating', 'bedrooms', 'beds', 'TV', 
                'zipcode'], axis=1)

  X["zipcode"] = X["zipcode"].replace("Near 91304", 91304)
  X["zipcode"] = X["zipcode"].replace("1m", 10023)
  X["zipcode"] = pd.to_numeric(X["zipcode"], errors="coerce")
  X["zipcode"] = X["zipcode"].replace(np.nan, X["zipcode"].median())
  X["zipcode"] = X["zipcode"].astype(str)
  X['zipcode'] = X['zipcode'][:5] 
  X["zipcode"] = X["zipcode"].replace(".", "")  
  X["zipcode"] = X["zipcode"].astype(float)

  X = X.fillna(0)

  encoder = ce.OrdinalEncoder()
  X = encoder.fit_transform(X)

  X = X.astype(float)

  return X

In [3]:
df = pd.read_csv('clean.csv')

FileNotFoundError: [Errno 2] File clean.csv does not exist: 'clean.csv'

In [10]:
df = wrangle(df)

In [11]:
train, test = train_test_split(df, test_size=0.2, random_state=42)


In [12]:
X_train = train.drop(columns="log_price")
y_train = train["log_price"]

X_test = test.drop(columns="log_price")
y_test = test["log_price"]

In [31]:
from tensorflow.keras.callbacks import EarlyStopping

adam = Adam(learning_rate=0.01)

stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)

model = Sequential([
    Dense(512, input_dim=10, activation= 'relu'),
    Dense(256, activation="relu"),
    Dense(128, activation="relu"),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
    ])

model.compile(loss='mean_squared_error', optimizer=adam, metrics=['mae'])

In [32]:
model.fit(x=X_train, 
          y=y_train, 
          epochs=50, 
          validation_data=(X_test, y_test))

Epoch 1/50
533/533 [==============================] - 3s 5ms/step - loss: 2337.7449 - mae: 1.4524 - val_loss: 0.7568 - val_mae: 0.5225
Epoch 2/50
533/533 [==============================] - 2s 5ms/step - loss: 14.9625 - mae: 0.4596 - val_loss: 12.7529 - val_mae: 0.6010
Epoch 3/50
533/533 [==============================] - 3s 5ms/step - loss: 0.3019 - mae: 0.4169 - val_loss: 8.1485 - val_mae: 1.0499
Epoch 4/50
533/533 [==============================] - 2s 5ms/step - loss: 0.2940 - mae: 0.4101 - val_loss: 0.2656 - val_mae: 0.3879
Epoch 5/50
533/533 [==============================] - 2s 5ms/step - loss: 0.2833 - mae: 0.4053 - val_loss: 0.4033 - val_mae: 0.4844
Epoch 6/50
533/533 [==============================] - 2s 4ms/step - loss: 0.2718 - mae: 0.3973 - val_loss: 0.3475 - val_mae: 0.4495
Epoch 7/50
533/533 [==============================] - 2s 5ms/step - loss: 0.2642 - mae: 0.3905 - val_loss: 0.2445 - val_mae: 0.3709
Epoch 8/50
533/533 [==============================] - 3s 5ms/step - los

In [33]:
kristine_models = model.save('kristine_model_1')

INFO:tensorflow:Assets written to: kristine_model_1/assets


In [34]:
test_model_1 = tf.keras.models.load_model('kristine_model_1')

In [35]:
test_model

In [36]:
test_model_1.predict(X_test)

array([[4.3169956],
       [4.319581 ],
       [5.062414 ],
       ...,
       [5.345599 ],
       [5.9439096],
       [4.3998146]], dtype=float32)

In [ ]:
kristine_models = model.save('kristine_model_0')